In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
import os,numpy

#from sklearn import preprocessing

def _get_data_path():
# have corrected this code on the copy on nirgun
    return os.path.join(os.getcwd().split("code")[0],"data")

def get_data(scenario, **kwargs):
    
    assert scenario in ["/gdrive/My Drive/periodic_tf/data/all"]
  # print(scenario)

    X_2d = numpy.load(os.path.join(_get_data_path(), str(scenario), "X_2d.npy"))
    X_features = numpy.load(os.path.join(_get_data_path(), str(scenario), "X_features.npy"))
    y = numpy.load(os.path.join(_get_data_path(), str(scenario), "y.npy"))
    lcid = numpy.load(os.path.join(_get_data_path(), str(scenario), "lcid.npy"))
    indices = numpy.array(range(len(y)))

    if 'classes' in kwargs.keys():
	#only picking out the data corresponding the classes provided
        mask = numpy.in1d(y, numpy.array(kwargs['classes']))
        y = y[mask]
        X_2d = X_2d[mask]
        X_features = X_features[mask]
        indices = indices[mask]
        lcid = lcid[mask]

    # shuffle data
    if 'shuffle' in kwargs.keys():
        seed = 0
        if 'seed' in kwargs.keys():
            seed = kwargs['seed']
        numpy.random.seed(seed)
        perm = numpy.random.permutation(len(y))
        y = y[perm]
        X_2d = X_2d[perm]
        X_features = X_features[perm]
        indices = indices[perm]
        lcid = lcid[perm]
    
    return X_2d, X_features, y, indices, lcid
    


In [0]:
import os
import numpy
import pandas as pd
import matplotlib.pyplot as plt

def _colorbar_fmt(x, pos):
    fm = '% *d' % (5, x)
    return fm

def ensure_dir(f):
    
    d = os.path.dirname(f)
    if not os.path.exists(d):
        try:
            os.makedirs(d)
        except:
            pass
        
def plot_confusion(y_test, preds, ofname):
    import seaborn
    from sklearn import metrics
    # confusion matrix:
    # y_true = [2, 0, 2, 2, 0, 1]
    # y_pred = [0, 0, 2, 2, 0, 2]
    # array([[2, 0, 0],
    #        [0, 0, 1],
    #        [1, 0, 2]])
    # x-axis: predictions, y-axis: true values (e.g., 2 prediction, but 1 as true label)
    cm = metrics.confusion_matrix(y_test, preds)
    c = cm
    cm = cm.astype('float')

    for i in range(cm.shape[0]):
        cm[i,:] *= 100/numpy.sum(cm[i,:])
    cm = cm.astype('int')


    classes = sorted(list(set(y_test)))

    df_cm = pd.DataFrame(cm, index=classes, columns=classes)
    plt.figure(figsize = (10,10))
    seaborn.set(font_scale=3.0)
    seaborn.heatmap(df_cm, annot=True, fmt="d",  linewidths=.5,  cmap="YlGnBu", square=True, cbar=False)
    plt.xlabel("Prediction")
    plt.ylabel("True Class")
    plt.savefig(ofname, bbox_inches='tight')
    plt.close()
   
    df_c = pd.DataFrame(c, index=classes, columns=classes)
    plt.figure(figsize = (10,10))
    seaborn.set(font_scale=3.0)
    seaborn.heatmap(df_c, annot=True, fmt="d",  linewidths=.5,  cmap="YlGnBu", square=True, cbar=False)
    plt.xlabel("Prediction")
    plt.ylabel("True Class")
    plt.savefig("/gdrive/My Drive/periodic_tf/code/experiments/cnn/confusion_cnn.png", bbox_inches='tight')
    plt.close()


def plot_image(img, ofname, titles=None, figsize=(10,5)):
    """ Three sub-images given ...
    """
    
    fig = plt.figure(tight_layout=True, figsize=figsize)
    
    plt.imshow(img)
    plt.savefig(ofname, bbox_inches='tight')
    plt.close()
    
def plot_misclassifications(y, preds, X, orig_indices, odir, titles=None, verbose=0):
    
    orig_indices = numpy.array(orig_indices)
    
    misclassifications = numpy.array(range(len(y)))
    misclassifications = misclassifications[y != preds]
    misclassifications_indices = orig_indices[y != preds]
    
    if verbose > 0:
        print("Number of test elements: %i" % len(y))
        print("Misclassifications: %s" % str(misclassifications_indices))
        print("Plotting misclassifications ...")
        
    for i in range(len(misclassifications)):
        print("Plotting misclassification %i of %i ..." % (i, len(misclassifications)))
        index = misclassifications[i]
        orig_index = misclassifications_indices[i]
        
        ofname = os.path.join(odir, str(y[index]), str(orig_index) + "_" + str(preds[index]) + ".png")
        ensure_dir(ofname)
        plot_image(X[index], ofname, titles=titles)
         


In [0]:
import matplotlib
matplotlib.use('Agg')

import os
import copy
#import generate
import numpy
import tensorflow as tf
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from util import plot_confusion, plot_misclassifications
import numpy as np
from six.moves import cPickle
import pickle
from imblearn.over_sampling import SMOTE 
from keras.engine.topology import Input
from keras.optimizers import Adam
from keras import regularizers

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [0]:
# parameters
epochs = 500
learning_rate = 0.0002
verbose = 1
seed = 0
classes = [1,2,4,5,6,8,13]
#what are classes
	
test_size = 0.2

In [0]:
# get data and encode labels
X_2d, X_features, y, indices, lcid = get_data("/gdrive/My Drive/periodic_tf/data/all", classes=classes, shuffle=True, seed=seed)
#print(scenario)
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(y)
y = labelencoder.transform(y).astype(numpy.int32)
print("Total number of instances: " + str(len(y)))

Total number of instances: 45630


In [0]:
# split data (train/test)
X_train, X_test, y_train, y_test, lcid_train, lcid_test, indices_train, indices_test = train_test_split(X_2d, y, lcid, indices, test_size=test_size, random_state=seed)
X_test_plot = copy.deepcopy(X_test)
# why reshaping ?
# X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]))
# X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]))
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
print("Number of training instances: %i" % len(y_train))
print("Number of test instances: %i" % len(y_test))

Number of training instances: 36504
Number of test instances: 9126


In [0]:
X_train.shape

(36504, 22, 24, 1)

In [0]:
tf.train.AdamOptimizer(
    learning_rate=0.0002,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='adam'
)

In [0]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=(22,24,1)))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation="relu", 
                 #data_format = "channels_first",
                 kernel_regularizer=regularizers.l2(0.01), 
                 #input_shape=(23,24,1),
                 padding='valid',
                 name="conv2d1"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid', 
                 #data_format = "channels_first",
                 name="maxpool2d1",
                 strides=(2,2)))
#model.add(Dropout(rate=0.1))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), activation="relu", 
                 kernel_regularizer=regularizers.l2(0.01),
                 padding='valid', 
                 #data_format = "channels_first",
                 name="conv2d2"))
#model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Flatten(
    #data_format = "channels_first", 
    name="flatten"))
#model.add(Dense(units=512, kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(rate=0.5))
#model.add(Dense(units=512, kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(units=7, activation="softmax", name="preds"))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "/gdrive/My Drive/periodic_tf/code/experiments/training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model.save_weights(checkpoint_path.format(epoch=0))
model.fit(X_train, y_train, epochs=500, callbacks = [cp_callback])
model.evaluate(X_test, y_test)

W0801 13:08:40.585912 140179752503168 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0801 13:08:40.828938 140179752503168 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d1 (Conv2D)             (None, 18, 20, 16)        416       
_________________________________________________________________
maxpool2d1 (MaxPooling2D)    (None, 9, 10, 16)         0         
_________________________________________________________________
conv2d2 (Conv2D)             (None, 5, 6, 64)          25664     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
preds (Dense)                (None, 7)                 13447     
Total params: 39,527
Trainable params: 39,527
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
36504/36504 [==============================] - 9s 251us/sample - loss: 0.7931 - acc: 0.7574
Epoch 2/500
36

[0.5195589505892011, 0.83552486]

In [0]:
preds = model.predict_classes(X_test, verbose=1)
preds_proba = model.predict(X_test, verbose=1)

y_test = labelencoder.inverse_transform(y_test)
preds = labelencoder.inverse_transform(preds)
y_train = labelencoder.inverse_transform(y_train)

9126/9126 [==============================] - 1s 72us/sample


In [0]:
print("F1 Score: "+str(f1_score(y_test.reshape(y_test.shape[0]),preds.reshape(preds.shape[0]),average=None)))
print("Matthews correlation coefficient (MCC): "+str(matthews_corrcoef(y_test.reshape(y_test.shape[0]),preds.reshape(preds.shape[0]))))

F1 Score: [0.90025038 0.80480824 0.6637931  0.71272115 0.15652174 0.33755274
 0.90987124]
Matthews correlation coefficient (MCC): 0.6797470325190401


In [0]:
# plot misclassifications
plot_misclassifications(y_test, preds, X_test_plot, indices_test, "/gdrive/My Drive/periodic_tf/code/experiments/cnn/misclassifications")

Plotting misclassification 0 of 1501 ...


/usr/local/lib/python3.6/dist-packages/matplotlib/figure.py:2369: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


Plotting misclassification 1 of 1501 ...
Plotting misclassification 2 of 1501 ...
Plotting misclassification 3 of 1501 ...
Plotting misclassification 4 of 1501 ...
Plotting misclassification 5 of 1501 ...
Plotting misclassification 6 of 1501 ...
Plotting misclassification 7 of 1501 ...
Plotting misclassification 8 of 1501 ...
Plotting misclassification 9 of 1501 ...
Plotting misclassification 10 of 1501 ...
Plotting misclassification 11 of 1501 ...
Plotting misclassification 12 of 1501 ...
Plotting misclassification 13 of 1501 ...
Plotting misclassification 14 of 1501 ...
Plotting misclassification 15 of 1501 ...
Plotting misclassification 16 of 1501 ...
Plotting misclassification 17 of 1501 ...
Plotting misclassification 18 of 1501 ...
Plotting misclassification 19 of 1501 ...
Plotting misclassification 20 of 1501 ...
Plotting misclassification 21 of 1501 ...
Plotting misclassification 22 of 1501 ...
Plotting misclassification 23 of 1501 ...
Plotting misclassification 24 of 1501 ...
P

In [0]:
# save output
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/X_test",X_test)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/y_test",y_test)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/X_train",X_train)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/y_train",y_train)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/lcid_train",lcid_train)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/lcid_test",lcid_test)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/indices_train",indices_train)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/indices_test",indices_test)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/preds_proba",preds_proba)
numpy.save("/gdrive/My Drive/periodic_tf/code/experiments/cnn/preds",preds)
numpy.savetxt("/gdrive/My Drive/periodic_tf/code/experiments/cnn/y_test_cnn.csv", y_test, delimiter=",", fmt='%.4f')
numpy.savetxt("/gdrive/My Drive/periodic_tf/code/experiments/cnn/preds_cnn.csv", preds, delimiter=",", fmt='%.4f')
numpy.savetxt("/gdrive/My Drive/periodic_tf/code/experiments/cnn/preds_proba_cnn.csv", preds_proba, delimiter=",", fmt='%.4f')
plot_confusion(y_test, preds, "/gdrive/My Drive/periodic_tf/code/experiments/cnn/confusion_cnn_hpercent.png")

In [0]:
tf.keras.models.save_model(
    model,
    "/gdrive/My Drive/periodic_tf/code/experiments/cnn/model.h5",
    overwrite=True,
    include_optimizer=True
)